In [1]:
import torch
import torchvision
import os.path

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from sklearn import manifold
from math import exp, sqrt
from torch.autograd import Variable
from my_dataset import MNIST_M
from my_dataset import ST_Dataset

%matplotlib inline
%load_ext skip_kernel_extension

import torch.utils.model_zoo as model_zoo

from data_loader import get_train_test_loader, get_office31_dataloader

use_gpu = torch.cuda.is_available()
print("use_gpu = " + str(use_gpu))

def reset_seq(seq):
    for m in seq:
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, sqrt(2 / n))
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            size = m.weight.size()
            fan_out = size[0] # number of rows
            fan_in = size[1] # number of columns
            m.weight.data.normal_(0, sqrt(2 / (fan_in + fan_out)))
            m.bias.data.zero_()
        elif hasattr(m, 'reset_parameters'):
            m.reset_parameters()
            
def evaluate_da_accuracy(model, dataloader, source):
    model.eval()
    correct_LC = 0
    correct_DC = 0
    total = 0
    
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            inputs, labels = data
            if (use_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()
            inputs, labels = Variable(inputs), Variable(labels)
            outputs_LC, outputs_DC = model(inputs)
            correct_LC += (torch.max(outputs_LC.data, 1)[1] == labels.data).sum().item()
            if source:
                correct_DC += labels.size()[0] - outputs_DC.data.sum().item()
            else:
                correct_DC += outputs_DC.data.sum().item()
            total += labels.size()[0]
        acc_LC = correct_LC / total
        acc_DC = correct_DC / total
    return acc_LC, acc_DC

use_gpu = True


In [2]:
trainloader_source = get_office31_dataloader("amazon", batch_size=102)

[INFO] Loading datasets: amazon


In [3]:
trainloader_target = get_office31_dataloader("webcam", batch_size=26)

[INFO] Loading datasets: webcam


In [4]:
class GRL_func(torch.autograd.Function):

    @staticmethod
    def forward(ctx, inputs, lamda):
        ctx.save_for_backward(lamda)
        return inputs

    @staticmethod
    def backward(ctx, grad_outputs):
        lamda, = ctx.saved_tensors
        return -lamda * grad_outputs, None

class GRL(nn.Module):
    
    def __init__(self, lamda_init):
        super(GRL, self).__init__()
        self.GRL_func = GRL_func.apply
        self.lamda = nn.Parameter(torch.Tensor(1), requires_grad=False)
        self.set_lamda(lamda_init)
        
    def forward(self, x):
        return self.GRL_func(x, self.lamda)
    
    def set_lamda(self, lamda_new):
        self.lamda[0] = lamda_new

In [5]:
class AlexNet_DA(nn.Module):

    def __init__(self, lamda_init):
        super(AlexNet_DA, self).__init__()
        # lambda
        self.lamda = lamda_init
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 256),
            nn.ReLU(inplace=True)
        )
        self.label_classifier = nn.Sequential(
            nn.Linear(256, 31)
        )
        self.GRL_layer = GRL(lamda_init)
        self.domain_classifier = nn.Sequential(
            nn.Linear(256, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        x_l = self.label_classifier(x)
        x_d = self.GRL_layer(x)
        x_d = self.domain_classifier(x_d)
        return x_l, x_d
    
    def set_lamda(self, lamda_new):
        self.GRL_layer.set_lamda(lamda_new)
    
    def load_pretrained_part(self, state_dict):
        own_state = self.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                print(name)
                continue
            if isinstance(param, nn.Parameter):
                # backwards compatibility for serialized parameters
                param = param.data
            own_state[name].copy_(param)

In [6]:
cnn_da = AlexNet_DA(0)

model_url = "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth"

cnn_da.load_pretrained_part(model_zoo.load_url(model_url))

if (use_gpu):
    cnn_da.cuda()
print(cnn_da)

classifier.6.weight
classifier.6.bias
AlexNet_DA(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    

In [23]:
for param in cnn_da.features.parameters():
    param.requires_grad = False

lr_init = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_da.parameters()), lr=lr_init, momentum=0.9)

def adjust_lr(optimizer, p):
    global lr_init
    lr_0 = lr_init
    alpha = 10
    beta = 0.75
    lr = lr_0 / (1 + alpha * p) ** beta
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [24]:
prev_loss = np.float("inf")
total_epoch = 40
finetune_epoch_start = 20
reset_seq(cnn_da.classifier)
reset_seq(cnn_da.label_classifier)

early_stop_cnt = 0

for epoch in range(total_epoch):
    epoch_loss = 0.0
    running_loss = 0.0
    p = epoch * 1.0 / total_epoch
    lr = adjust_lr(optimizer, p)
    if epoch == finetune_epoch_start:
        for param in cnn_da.features.parameters():
            param.requires_grad = True
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_da.parameters()), lr=lr, momentum=0.9)
    for i, data in enumerate(trainloader_source):
        inputs, labels = data
        if (use_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()
        outputs_LC, _ = cnn_da(inputs)
        loss = criterion(outputs_LC, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        running_loss += loss.item()
        if i % 2 == 1:    # print every 50 mini-batches
            print('[%2d] batch loss: %.3f' %
                  (i + 1, running_loss / 2))
            running_loss = 0.0
    print("epoch %d loss: %.3f -> %.3f\n" % (epoch + 1, prev_loss, epoch_loss))
    if prev_loss - epoch_loss < 0.1:
        prev_loss = epoch_loss
        if epoch > finetune_epoch_start:
            early_stop_cnt += 1
            if early_stop_cnt > 2:
                break
    else:
        early_stop_cnt = 0
        prev_loss = epoch_loss

[ 2] batch loss: 6.873
[ 4] batch loss: 5.648
[ 6] batch loss: 4.102
[ 8] batch loss: 3.550
[10] batch loss: 3.385
[12] batch loss: 3.214
[14] batch loss: 3.267
[16] batch loss: 3.199
[18] batch loss: 3.106
[20] batch loss: 3.185
[22] batch loss: 2.953
[24] batch loss: 2.974
[26] batch loss: 2.832
[28] batch loss: 2.536
epoch 1 loss: inf -> 101.650

[ 2] batch loss: 2.573
[ 4] batch loss: 2.658
[ 6] batch loss: 2.388
[ 8] batch loss: 2.459
[10] batch loss: 2.232
[12] batch loss: 2.156
[14] batch loss: 2.064
[16] batch loss: 2.219
[18] batch loss: 2.023
[20] batch loss: 1.902
[22] batch loss: 1.771
[24] batch loss: 1.908
[26] batch loss: 1.924
[28] batch loss: 1.637
epoch 2 loss: 101.650 -> 59.829

[ 2] batch loss: 1.819
[ 4] batch loss: 1.434
[ 6] batch loss: 1.689
[ 8] batch loss: 1.657
[10] batch loss: 1.607
[12] batch loss: 1.493
[14] batch loss: 1.542
[16] batch loss: 1.434
[18] batch loss: 1.370
[20] batch loss: 1.589
[22] batch loss: 1.226
[24] batch loss: 1.169
[26] batch loss: 

[ 8] batch loss: 0.295
[10] batch loss: 0.204
[12] batch loss: 0.201
[14] batch loss: 0.187
[16] batch loss: 0.203
[18] batch loss: 0.210
[20] batch loss: 0.235
[22] batch loss: 0.225
[24] batch loss: 0.293
[26] batch loss: 0.197
[28] batch loss: 0.226
epoch 24 loss: 7.238 -> 6.328

[ 2] batch loss: 0.277
[ 4] batch loss: 0.208
[ 6] batch loss: 0.246
[ 8] batch loss: 0.192
[10] batch loss: 0.214
[12] batch loss: 0.205
[14] batch loss: 0.222
[16] batch loss: 0.191
[18] batch loss: 0.141
[20] batch loss: 0.146
[22] batch loss: 0.199
[24] batch loss: 0.168
[26] batch loss: 0.301
[28] batch loss: 0.218
epoch 25 loss: 6.328 -> 5.859

[ 2] batch loss: 0.183
[ 4] batch loss: 0.097
[ 6] batch loss: 0.167
[ 8] batch loss: 0.098
[10] batch loss: 0.175
[12] batch loss: 0.143
[14] batch loss: 0.271
[16] batch loss: 0.153
[18] batch loss: 0.171
[20] batch loss: 0.195
[22] batch loss: 0.190
[24] batch loss: 0.148
[26] batch loss: 0.148
[28] batch loss: 0.172
epoch 26 loss: 5.859 -> 4.621

[ 2] batch

In [25]:
evaluate_da_accuracy(cnn_da, trainloader_source, source=True)

(1.0, 0.4629394645000291)

In [26]:
evaluate_da_accuracy(cnn_da, trainloader_target, source=False)

(0.49559748427672956, 0.5306384944315976)

In [10]:
from math import exp

lr_init = 0.01
criterion_LC = nn.CrossEntropyLoss()
criterion_DC = nn.BCELoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_da.parameters()), lr=lr_init, momentum=0.9)

def adjust_lr(optimizer, p):
    global lr_init
    lr_0 = lr_init
    alpha = 10
    beta = 0.75
    lr = lr_0 / (1 + alpha * p) ** beta
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def adjust_lamda(model, p):
    gamma = 10
    lamda = 2 / (1 + exp(- gamma * p)) - 1
    model.set_lamda(lamda)
    return lamda

In [7]:
cnn_da.load_pretrained_part(torch.load("./parameters/cnn_amazon.pt"))

In [8]:
for param in cnn_da.features.parameters():
    param.requires_grad = True

In [ ]:
prev_loss = np.float("inf")
prev_loss_LC = np.float("inf")
prev_loss_DC = np.float("inf")
total_epoch = 100
reset_seq(cnn_da.domain_classifier)

for epoch in range(total_epoch):
    epoch_loss = 0.0
    epoch_loss_LC = 0.0
    epoch_loss_DC = 0.0
    running_loss = 0.0
    p = epoch * 1.0 / total_epoch
    lr = adjust_lr(optimizer, p)
    dslr_iter = iter(trainloader_source)
    webcam_iter = iter(trainloader_target)
    i = 0
    while True:
        try:
            images_s, labels_s = dslr_iter.next()
            images_t, labels_t = webcam_iter.next()
        except:
            break
        inputs, labels = torch.cat((images_s, images_t)), torch.cat((labels_s, labels_t))
        source_size, target_size = labels_s.size(0), labels_t.size(0)
        domains = torch.cat((torch.zeros(source_size), torch.ones(target_size)))
        if (use_gpu):
            inputs, labels, domains = inputs.cuda(), labels.cuda(), domains.cuda()
        inputs, labels, domains = Variable(inputs), Variable(labels), Variable(domains)
        optimizer.zero_grad()
        outputs_LC, outputs_DC = cnn_da(inputs)
        loss_LC = criterion_LC(outputs_LC[:source_size], labels[:source_size])
        loss_DC = criterion_DC(outputs_DC.view(-1), domains)
        loss = loss_LC + loss_DC
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_loss_LC += loss_LC.item()
        epoch_loss_DC += loss_DC.item()
        running_loss += loss.item()
        if i % 2 == 1:    # print every 50 mini-batches
            print('[%2d] batch loss: %.3f' %
                  (i + 1, running_loss / 2))
            running_loss = 0.0
        i += 1
    print("epoch %d loss: %.3f -> %.3f" % (epoch + 1, prev_loss, epoch_loss))
    print("LC loss: %.3f -> %.3f" % (prev_loss_LC, epoch_loss_LC))
    print("DC loss: %.3f -> %.3f" % (prev_loss_DC, epoch_loss_DC))
    if epoch % 10 == 9:
        acc_l, acc_d = evaluate_da_accuracy(cnn_da, trainloader_target, source=False)
        print(acc_l, acc_d)
    print()
    prev_loss = epoch_loss
    prev_loss_LC = epoch_loss_LC
    prev_loss_DC = epoch_loss_DC

[ 2] batch loss: 2.555
[ 4] batch loss: 3.311
[ 6] batch loss: 1.788
[ 8] batch loss: 1.136
[10] batch loss: 1.180
[12] batch loss: 1.125
[14] batch loss: 1.315
[16] batch loss: 1.094
[18] batch loss: 1.525
[20] batch loss: 1.145
[22] batch loss: 1.231
[24] batch loss: 1.258
[26] batch loss: 1.413
[28] batch loss: 1.773
epoch 1 loss: inf -> 43.699
LC loss: inf -> 18.238
DC loss: inf -> 25.461

[ 2] batch loss: 1.577
[ 4] batch loss: 1.207
[ 6] batch loss: 1.312
[ 8] batch loss: 1.251
[10] batch loss: 1.217
[12] batch loss: 1.098
[14] batch loss: 1.108
[16] batch loss: 1.194
[18] batch loss: 1.186
[20] batch loss: 1.127
[22] batch loss: 1.336
[24] batch loss: 1.252
[26] batch loss: 1.158
[28] batch loss: 1.309
epoch 2 loss: 43.699 -> 34.664
LC loss: 18.238 -> 21.686
DC loss: 25.461 -> 12.979

[ 2] batch loss: 1.082
[ 4] batch loss: 1.038
[ 6] batch loss: 0.913
[ 8] batch loss: 0.765
[10] batch loss: 0.991
[12] batch loss: 1.047
[14] batch loss: 0.907
[16] batch loss: 0.994
[18] batch lo

[10] batch loss: 0.105
[12] batch loss: 0.109
[14] batch loss: 0.078
[16] batch loss: 0.104
[18] batch loss: 0.079
[20] batch loss: 0.082
[22] batch loss: 0.074
[24] batch loss: 0.105
[26] batch loss: 0.083
[28] batch loss: 0.105
epoch 21 loss: 2.583 -> 2.448
LC loss: 0.006 -> 0.006
DC loss: 2.577 -> 2.442

[ 2] batch loss: 0.059
[ 4] batch loss: 0.084
[ 6] batch loss: 0.068
[ 8] batch loss: 0.085
[10] batch loss: 0.068
[12] batch loss: 0.093
[14] batch loss: 0.114
[16] batch loss: 0.067
[18] batch loss: 0.078
[20] batch loss: 0.059
[22] batch loss: 0.078
[24] batch loss: 0.063
[26] batch loss: 0.082
[28] batch loss: 0.082
epoch 22 loss: 2.448 -> 2.159
LC loss: 0.006 -> 0.005
DC loss: 2.442 -> 2.154

[ 2] batch loss: 0.053
[ 4] batch loss: 0.072
[ 6] batch loss: 0.060
[ 8] batch loss: 0.079
[10] batch loss: 0.055
[12] batch loss: 0.076
[14] batch loss: 0.077
[16] batch loss: 0.075
[18] batch loss: 0.069
[20] batch loss: 0.068
[22] batch loss: 0.073
[24] batch loss: 0.095
[26] batch los

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fd1b06dea20>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/opt/conda/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/opt/conda/lib/python3.6/multiprocessing/reducti

[ 2] batch loss: 0.042
[ 4] batch loss: 0.051
[ 6] batch loss: 0.093
[ 8] batch loss: 0.086
[10] batch loss: 0.044
[12] batch loss: 0.047
[14] batch loss: 0.061
[16] batch loss: 0.048
[18] batch loss: 0.064
[20] batch loss: 0.064
[22] batch loss: 0.042
[24] batch loss: 0.058
[26] batch loss: 0.037
[28] batch loss: 0.055
epoch 30 loss: 1.520 -> 1.585
LC loss: 0.004 -> 0.004
DC loss: 1.516 -> 1.581
0.4427672955974843 0.8865449041690466

[ 2] batch loss: 0.071
[ 4] batch loss: 0.031
[ 6] batch loss: 0.042
[ 8] batch loss: 0.044
[10] batch loss: 0.060
[12] batch loss: 0.038
[14] batch loss: 0.036
[16] batch loss: 0.036
[18] batch loss: 0.050
[20] batch loss: 0.064
[22] batch loss: 0.066
[24] batch loss: 0.036
[26] batch loss: 0.038
[28] batch loss: 0.087
epoch 31 loss: 1.585 -> 1.399
LC loss: 0.004 -> 0.003
DC loss: 1.581 -> 1.395

[ 2] batch loss: 0.040
[ 4] batch loss: 0.047
[ 6] batch loss: 0.050
[ 8] batch loss: 0.041
[10] batch loss: 0.043
[12] batch loss: 0.104
[14] batch loss: 0.040

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fd1b06de9b0>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/opt/conda/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/opt/conda/lib/python3.6/multiprocessing/reducti

...

[ 2] batch loss: 0.004
[ 4] batch loss: 0.006
[ 6] batch loss: 0.007
[ 8] batch loss: 0.007
[10] batch loss: 0.005
[12] batch loss: 0.005
[14] batch loss: 0.007
[16] batch loss: 0.006
[18] batch loss: 0.005
[20] batch loss: 0.005
[22] batch loss: 0.005
[24] batch loss: 0.006
[26] batch loss: 0.008
[28] batch loss: 0.006
epoch 65 loss: 0.180 -> 0.164
LC loss: 0.002 -> 0.002
DC loss: 0.178 -> 0.162

[ 2] batch loss: 0.007
[ 4] batch loss: 0.005
[ 6] batch loss: 0.005
[ 8] batch loss: 0.008
[10] batch loss: 0.006
[12] batch loss: 0.005
[14] batch loss: 0.006
[16] batch loss: 0.005
[18] batch loss: 0.003
[20] batch loss: 0.009
[22] batch loss: 0.006
[24] batch loss: 0.007
[26] batch loss: 0.004
[28] batch loss: 0.007
epoch 66 loss: 0.164 -> 0.168
LC loss: 0.002 -> 0.002
DC loss: 0.162 -> 0.166

[ 2] batch loss: 0.004
[ 4] batch loss: 0.009
[ 6] batch loss: 0.007
[ 8] batch loss: 0.006
[10] batch loss: 0.005
[12] batch loss: 0.005
[14] batch loss: 0.010
[16] batch loss: 0.002
[18] batch los

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fd21cdf30b8>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process

[ 2] batch loss: 0.008
[ 4] batch loss: 0.005
[ 6] batch loss: 0.003
[ 8] batch loss: 0.004
[10] batch loss: 0.006
[12] batch loss: 0.005
[14] batch loss: 0.007
[16] batch loss: 0.006
[18] batch loss: 0.006
[20] batch loss: 0.003
[22] batch loss: 0.005
[24] batch loss: 0.006
[26] batch loss: 0.005
[28] batch loss: 0.007
epoch 72 loss: 0.169 -> 0.150
LC loss: 0.002 -> 0.002
DC loss: 0.168 -> 0.149

[ 2] batch loss: 0.005
[ 4] batch loss: 0.007
[ 6] batch loss: 0.005
[ 8] batch loss: 0.005
[10] batch loss: 0.004
[12] batch loss: 0.005
[14] batch loss: 0.005
[16] batch loss: 0.007
[18] batch loss: 0.003
[20] batch loss: 0.004
[22] batch loss: 0.006
[24] batch loss: 0.008
[26] batch loss: 0.006
[28] batch loss: 0.005
epoch 73 loss: 0.150 -> 0.151
LC loss: 0.002 -> 0.002
DC loss: 0.149 -> 0.150

[ 2] batch loss: 0.005
[ 4] batch loss: 0.003
[ 6] batch loss: 0.007
[ 8] batch loss: 0.003
[10] batch loss: 0.004
[12] batch loss: 0.005
[14] batch loss: 0.004
[16] batch loss: 0.006
[18] batch los

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fd1b06e7320>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/opt/conda/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_handle
    return recvfds(s, 1)[0]
  File "/opt/conda/lib/python3.6/multiprocessing/reducti

[ 2] batch loss: 0.004
[ 4] batch loss: 0.006
[ 6] batch loss: 0.004
[ 8] batch loss: 0.003
[10] batch loss: 0.005
[12] batch loss: 0.003
[14] batch loss: 0.004
[16] batch loss: 0.006
[18] batch loss: 0.003
[20] batch loss: 0.007
[22] batch loss: 0.004
[24] batch loss: 0.004
[26] batch loss: 0.005
[28] batch loss: 0.003
epoch 77 loss: 0.128 -> 0.122
LC loss: 0.002 -> 0.002
DC loss: 0.126 -> 0.120

[ 2] batch loss: 0.003
[ 4] batch loss: 0.004
[ 6] batch loss: 0.004
[ 8] batch loss: 0.004
[10] batch loss: 0.005
[12] batch loss: 0.003
[14] batch loss: 0.004
[16] batch loss: 0.005
[18] batch loss: 0.005
[20] batch loss: 0.003
[22] batch loss: 0.004
[24] batch loss: 0.008
[26] batch loss: 0.004
[28] batch loss: 0.004
epoch 78 loss: 0.122 -> 0.115
LC loss: 0.002 -> 0.002
DC loss: 0.120 -> 0.113

[ 2] batch loss: 0.005
[ 4] batch loss: 0.004
[ 6] batch loss: 0.004
[ 8] batch loss: 0.004
[10] batch loss: 0.004
[12] batch loss: 0.002
[14] batch loss: 0.004
[16] batch loss: 0.002
[18] batch los

In [33]:
#torch.save(cnn_da.state_dict(), "./parameters/cnn_webcam_to_dslr_0.989.pt")

In [39]:
#cnn_da.load_state_dict(torch.load("./parameters/cnn_dslr_to_webcam.pt"))

In [ ]:
for p 